In [1]:
from bs4 import BeautifulSoup
import requests
import urllib
import json
import pandas as pd
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
def scrape(url):
    req = requests.get(url)
    return url, req.text

In [3]:
pagina="https://www.wikiart.org"

In [4]:
soup=BeautifulSoup(scrape(pagina+"/en/paintings-by-style")[1])

In [5]:
styles=soup.find_all('ul',{"class":"dictionaries-list"})[0]
styles=styles.find_all('li')

In [6]:
current_style=''
pager_links=[]
styles_links=[]
for li in range(len(styles)):
    if styles[li].find_all("a",href=True)==[]:
        current_style=styles[li].text.strip()
    else:
        pager_links.append(styles[li].find_all("a",href=True)[0]["href"])
        styles_links.append(current_style)

In [7]:
soup_images= [BeautifulSoup(scrape(pagina+x)[1]) for x in pager_links]

In [8]:
images_links = [x.find_all('main') for x in soup_images]

In [9]:
images_substyle = [x.find_all('h1')[0].text.strip() for x in soup_images]

In [10]:
links=["https:"+str(x[0]).split("\n")[3].split(",")[1].split(":")[2][:-2] for x in images_links]

In [11]:
diccionario_wikiarts=[]
for i,link in enumerate(links):
    open_link = urllib.request.urlopen(link)
    archivo = json.loads(open_link.read())
    archivo["style"]=styles_links[i]
    archivo["substyle"]=images_substyle[i]
    diccionario_wikiarts.append(archivo)

In [12]:
pd.to_pickle(diccionario_wikiarts,"/home/uriel/itam/semestre1/mineria_y_analisis_de_datos/wikiart/dict_wikiarts.pkl")

In [13]:
suma=0
for i in diccionario_wikiarts:
    imagenes=i["Paintings"]
    for j in  imagenes:
        imagen=j["image"]
        suma=suma+1
print(suma)

10144


In [ ]:
for i in diccionario_wikiarts:
    imagenes=i["Paintings"]
    for j in  imagenes:
        imagen=j["image"]
        id_=j["id"]
        obtener = requests.get(imagen,stream=True)
        with open("/home/uriel/itam/semestre1/mineria_y_analisis_de_datos/wikiart/" + id_ +".jpg", 'wb') as imagen:
            imagen.write(obtener.content)

**Ya tengo las primeras 224 imágenes del diccionario**